#### Modify Alpha-Beta Acid Batches

In [ ]:
import pandas as pd
import re
import os
import logging
import sys

# Set up logging
LOG_FILE = "log_alpha_beta.txt"
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, 
                    format="%(asctime)s - %(levelname)s - %(message)s")

# Define folder paths and dictionary file
individual_batches_path = "Data/Alpha_Beta_Acid/Individual Batches"
modified_batches_path = "Data/Alpha_Beta_Acid/Output"
dictionary_file_path = "Data/Alpha_Beta_Acid/Dictionary.xlsx"

def load_name_mapping(dictionary_file_path):
    """
    Load the dictionary Excel file and create a mapping of alternative names to standardized names.
    Arguments:
        dictionary_file_path (str): File path to the dictionary Excel file.
    Returns:
        dict: A dictionary where keys are alternative names and values are standardized names.
    Raises:
        FileNotFoundError: If dictionary file is missing
        Exception: If dictionary loading fails
    """
    try:
        df_dict = pd.read_excel(dictionary_file_path, header=None)
        name_mapping = {}
        for _, row in df_dict.iterrows():
            standardized_name = row[0]
            variations = row[1:].dropna().tolist()
            for variation in variations:
                name_mapping[variation] = standardized_name
        logging.info("Successfully loaded dictionary from %s", dictionary_file_path)
        return name_mapping
    except FileNotFoundError:
        logging.error("Dictionary file not found: %s", dictionary_file_path)
        sys.exit("Error: Dictionary file not found.")
    except Exception as e:
        logging.error("Error loading dictionary: %s", str(e))
        sys.exit("Error: Failed to load dictionary.")

def extract_repetition(value):
    """
    Extract the repetition number from a string if it ends with '_1', '_2', etc.
    Arguments:
        value (str): The input string containing a potential repetition pattern.
    Returns:
        str or None: The extracted repetition number if found, otherwise None.
    """
    match = re.search(r'_(\d+)(?=\s|$)', str(value))
    return match.group(1) if match else None

def extract_dilution_factor(value):
    """
    Extract the dilution factor from a string if it contains a number followed by 'x'.
    Arguments:
        value (str): The input string containing a potential dilution factor.
    Returns:
        str or None: The extracted dilution factor if found, otherwise None.
    """
    match = re.search(r'(\d+)(?=x)', str(value))
    return match.group(1) if match else None

def requires_dilution_factor(value):
    """
    Determine if a given value contains a dilution factor pattern.
    Arguments:
        value (str): The input string to check.
    Returns:
        bool: True if a dilution factor pattern is detected, otherwise False.
    """
    return bool(re.search(r'\d+x', str(value)))

def clean_data(df, name_mapping):
    """
    Clean and process the input DataFrame while retaining relevant columns and headers.
        - Extracts repetition numbers.
        - Extracts and handles dilution factors.
        - Standardizes compound names using the provided name mapping.
        - Removes unnecessary characters (e.g., repetition suffixes, dilution indicators).
        - Drops unnecessary columns based on missing values.
    Arguments:
        df (pd.DataFrame): The input DataFrame containing raw data.
        name_mapping (dict): Dictionary mapping alternative names to standardized names.
    Returns:
        pd.DataFrame: A cleaned DataFrame with processed columns.
    Raises:
        Exception: If the data cleaning process failed
    """
    try:
        df = df.iloc[:, 1:8]  # Keep columns B:H
        headers = df.iloc[0]  # Save headers
        df = df[1:].reset_index(drop=True)  # Remove first row
        df.columns = headers  # Assign headers
        first_column = df.columns[0]
        
        df['Repetition'] = df[first_column].apply(extract_repetition)  # Extract repetition
        df['Dilution Factor'] = df[first_column].apply(extract_dilution_factor)  # Extract dilution factor

        # Check if dilution factor column is needed
        df['Needs Dilution Factor'] = df[first_column].apply(requires_dilution_factor)
        
        df[first_column] = df[first_column].apply(lambda x: re.sub(r'_(\d+)(?=\s|$)', '', str(x)))  # Remove suffix
        df[first_column] = df[first_column].apply(lambda x: re.sub(r' (\d+)x', '', str(x)))  # Remove dilution part
        df[first_column] = df[first_column].apply(lambda x: name_mapping.get(str(x), str(x)))  # Standardize names
        
        # Rename the first column to "Compound Name"
        df = df.rename(columns={first_column: "Compound Name"})
        
        if df['Needs Dilution Factor'].any():  # If any row requires dilution factor
            df.insert(1, 'Repetition', df.pop('Repetition'))
            df.insert(2, 'Dilution Factor', df.pop('Dilution Factor'))
        else:
            df.insert(1, 'Repetition', df.pop('Repetition'))
            df = df.drop(columns=['Dilution Factor'])
        
        df = df.drop(columns=['Needs Dilution Factor'])
        
        # Drop rows with more than 3 missing values
        df = df.dropna(thresh=len(df.columns) - 3, axis=0)

        logging.info("Successfully cleaned data.")
        return df
    except Exception as e:
        logging.error("Error cleaning data: %s", str(e))
        raise

def process_excel_files():
    """
    Process all Excel files in the specified folder.
        - Loads the name mapping from the dictionary file.
        - Iterates through each Excel file in the folder.
        - Reads and cleans data from each sheet.
        - Saves the cleaned data to a new output Excel file.
    Raises:
        Exception: If the processing failed
    """
    name_mapping = load_name_mapping(dictionary_file_path)
    files_processed = 0

    try:
        if not os.path.exists(modified_batches_path):
            os.makedirs(modified_batches_path)

        files = [f for f in os.listdir(individual_batches_path) if f.endswith('.xlsx')]
        if not files:
            logging.error("No Excel files found in %s", individual_batches_path)
            sys.exit("Error: No Excel files found.")

        for file_name in files:
            file_path = os.path.join(individual_batches_path, file_name)
            xls = pd.ExcelFile(file_path)

            for sheet_name in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=sheet_name, header=0)  # Keep headers
                df_cleaned = clean_data(df, name_mapping)
                output_file = os.path.join(modified_batches_path, f"Modified_{os.path.splitext(file_name)[0]}.xlsx")
                df_cleaned.to_excel(output_file, index=False)

                logging.info("Processed and saved: %s", output_file)
                print(f"Processed file saved as: {output_file}")
                files_processed += 1

        logging.info("Processing completed. Total files processed: %d", files_processed)
    except Exception as e:
        logging.error("Error processing files: %s", str(e))
        sys.exit("Error: Processing failed.")

# Run the function
process_excel_files()

Processed file saved as: Data/Alpha_Beta_Acid/Output\Modified_Batch 1.xlsx
Processed file saved as: Data/Alpha_Beta_Acid/Output\Modified_Batch 2.xlsx
Processed file saved as: Data/Alpha_Beta_Acid/Output\Modified_Batch 3.xlsx
Processed file saved as: Data/Alpha_Beta_Acid/Output\Modified_Batch 4.xlsx
Processed file saved as: Data/Alpha_Beta_Acid/Output\Modified_Batch 5.xlsx


#### Create Transposed Alpha-Beta Acid Batches

In [ ]:
import os
import pandas as pd
import logging
import sys

# Configure logging
LOG_FILE = "log_alpha_beta.txt"
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, 
                    format="%(asctime)s - %(levelname)s - %(message)s")

def process_alpha_beta_acid_files(modified_batches_path, output_alpha_beta_acid):
    """
    Process Excel files containing Alpha and Beta Acid data.
        - Reads input files from the specified folder.
        - Extracts relevant chemical components and concentrations.
        - Saves the processed data in an output folder.
    Arguments:
        modified_batches_path (str): Path to the folder containing input Excel files.
        output_alpha_beta_acid (str): Path to the folder where processed files will be saved.
    """
    try:
        # Check if input folder exists
        if not os.path.exists(modified_batches_path):
            logging.error("Input folder does not exist: %s", modified_batches_path)
            sys.exit(f"Input folder not found: {modified_batches_path}")  # Exit on error

        # Get list of all Excel files in the input folder
        files = [f for f in os.listdir(modified_batches_path) if f.endswith(('.xlsx', '.xls'))]
        
        if not files:
            logging.warning("No Excel files found in the input folder: %s", modified_batches_path)
            print("Warning: No Excel files found.")
            sys.exit("No Excel files found in the input folder. Exiting the system.")  # Exit if no files

        # Ensure output folder exists
        os.makedirs(output_alpha_beta_acid, exist_ok=True)
        
        # Process each file
        for file in files:
            file_path = os.path.join(modified_batches_path, file)
            
            try:
                # Read the Excel file
                df = pd.read_excel(file_path)
                
                # Check if required columns exist
                required_columns = ['Compound Name', 'Repetition', 'n - (+) Adhumulone (α-acid)', 
                                    'n - (+) Adlupulone (β-acid)', 'Cohumulone (α-acid)', 'Colupulone (β-acid)']
                missing_columns = [col for col in required_columns if col not in df.columns]
                if missing_columns:
                    logging.warning("Skipping file %s due to missing columns: %s", file, missing_columns)
                    continue
                
                # Check if 'Dilution Factor' column exists
                dilution_factor_column_exists = 'Dilution Factor' in df.columns
                
                # Create an empty list to store output rows
                output_rows = []
                
                # Iterate through each row of the input dataframe
                for _, row in df.iterrows():
                    sample_name = row['Compound Name']
                    repetition = row['Repetition']
                    
                    # Get the chemical components and concentrations
                    components = ['n - (+) Adhumulone (α-acid)', 'n - (+) Adlupulone (β-acid)', 
                                  'Cohumulone (α-acid)', 'Colupulone (β-acid)']
                    for component in components:
                        # Prepare each output row
                        output_row = {
                            'Sample Name': sample_name,
                            'Chemical Component': component,
                            'Repetition': repetition,
                            'Dilution Factor': row['Dilution Factor'] if dilution_factor_column_exists else '',
                            'Concentration': row.get(component, None)
                        }
                        output_rows.append(output_row)
                
                # Convert the output rows into a DataFrame
                output_df = pd.DataFrame(output_rows)
                
                # Define output file path
                output_file_path = os.path.join(output_alpha_beta_acid, f"Output_{file}")
                
                # Write the processed data to a new Excel file
                output_df.to_excel(output_file_path, index=False)
                logging.info("Processed and saved: %s", output_file_path)
                print(f"Processed file saved as: {output_file_path}")
                
            except Exception as e:
                logging.error("Error processing file %s: %s", file, str(e))
                print(f"Error processing {file}: {str(e)}")
                sys.exit(f"Error processing {file}. Exiting the system.")  # Exit on error
    
    except Exception as e:
        logging.critical("Fatal error during processing: %s", str(e))
        print(f"Fatal error: {str(e)}")
        sys.exit(f"Fatal error: {str(e)}. Exiting the system.")  # Exit on error

# Input and Output folder paths
modified_batches_path = 'Data/Alpha_Beta_Acid/Output'  
output_alpha_beta_acid = 'Data/Alpha_Beta_Acid/Database' 

# Call the function to process the files
process_alpha_beta_acid_files(modified_batches_path, output_alpha_beta_acid)

Processed file saved as: Data/Alpha_Beta_Acid/Database\Output_Modified_Batch 1.xlsx
Processed file saved as: Data/Alpha_Beta_Acid/Database\Output_Modified_Batch 2.xlsx
Processed file saved as: Data/Alpha_Beta_Acid/Database\Output_Modified_Batch 3.xlsx
Processed file saved as: Data/Alpha_Beta_Acid/Database\Output_Modified_Batch 4.xlsx
Processed file saved as: Data/Alpha_Beta_Acid/Database\Output_Modified_Batch 5.xlsx


#### Create Pivot table for Cannabis  

In [ ]:
import os
import pandas as pd
import logging
import sys

# Set up logging configuration
LOG_FILE = "log_canabis.txt"
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, 
                    format="%(asctime)s - %(levelname)s - %(message)s")

# Function to process each Excel file
def process_canabis_files(input_cannabis_folder, output_cannabis_folder):
    """
    Processes Excel files in the input folder by extracting relevant data and saving it to the output folder.
    Each file is read, cleaned, and saved in a new format with relevant details.
    Arguments:
        input_cannabis_folder (str): Path to the folder containing input Excel files.
        output_cannabis_folder (str): Path to the folder where the processed output will be saved.
    """
    try:
        # Check if the input folder exists
        if not os.path.exists(input_cannabis_folder):
            logging.error("Input folder does not exist: %s", input_cannabis_folder)
            sys.exit(f"Input folder not found: {input_cannabis_folder}")  # Exit on error
        
        # Check if the output folder exists, create it if it doesn't
        if not os.path.exists(output_cannabis_folder):
            os.makedirs(output_cannabis_folder)
            logging.info("Output folder created: %s", output_cannabis_folder)

        # Loop through all files in the input folder
        for file_name in os.listdir(input_cannabis_folder):
            if file_name.endswith(".xlsx") or file_name.endswith(".xls"):
                file_path = os.path.join(input_cannabis_folder, file_name)
                try:
                    # Load the Excel file and select the sheet named "Mean (%)"
                    df = pd.read_excel(file_path, sheet_name="Mean (%)")
                    
                    # Get sample names from the first row (excluding the first cell)
                    sample_names = df.columns[1:].tolist()  # Exclude the first column (Chemical Component)
                    
                    # Initialize an empty list to store the output data
                    output_data = []
                    
                    # Loop through each row in the dataframe
                    for _, row in df.iterrows():
                        # The first column is the Chemical Component
                        chemical_component = row.iloc[0]  # First column in the row is the component
                        
                        # Loop through each sample (excluding the first column which is the component)
                        for sample_name in sample_names:
                            concentration = row[sample_name]
                            
                            # Clean the sample name by removing "Mean" and stripping extra spaces
                            cleaned_sample_name = sample_name.replace("Mean", "").strip()
                            
                            # Skip rows where the concentration is 'LOD' or 'LOQ'
                            if pd.notna(concentration) and (concentration != 'LOD' and concentration != 'LOQ'):
                                # Add the row to output data
                                output_data.append({
                                    "Sample Name": cleaned_sample_name,
                                    "Chemical Component": chemical_component,
                                    "Concentration": concentration
                                })
                    
                    # Create the output DataFrame
                    output_df = pd.DataFrame(output_data)
                    
                    # Save the output DataFrame to a new Excel file
                    output_file_name = f"Output_{file_name}"
                    output_file_path = os.path.join(output_cannabis_folder, output_file_name)
                    output_df.to_excel(output_file_path, index=False)
                    
                    logging.info("Processed file saved as %s", output_file_name)
                    print(f"Processed file saved as {output_file_name}")
                    
                except Exception as e:
                    logging.error("Error processing file %s: %s", file_name, str(e))
                    print(f"Error processing file {file_name}. Check log for details.")
                    sys.exit(f"Error processing file {file_name}. Exiting the system.")  # Exit on error
    
    except Exception as e:
        logging.error("Error in processing files: %s", str(e))
        print("An error occurred. Check the log for details.")
        sys.exit("An error occurred. Exiting the system.")  # Exit on error

# Folder paths
input_cannabis_folder = 'Data/Cannabis/Input' 
output_cannabis_folder = 'Data/Cannabis/Database'  

# Run the processing function
process_canabis_files(input_cannabis_folder, output_cannabis_folder)

Processed file saved as Output_SolventExtraction_percentage_Dilution1.xlsx
Processed file saved as Output_SolventExtraction_percentage_Dilution2.xlsx


#### Create Pivot table for Terpens

In [ ]:
import os
import pandas as pd
import logging
import sys

# Set up logging configuration
LOG_FILE = "log_terpenes.txt"
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, 
                    format="%(asctime)s - %(levelname)s - %(message)s")

# Function to process each Excel file
def process_terpens_files(input_terpenes_folder, output_terpenes_folder):
    """
    Processes Excel files in the input folder by extracting relevant data and saving it to the output folder.
    Each file is read, cleaned, and saved in a new format with relevant details.
    Arguments:
        input_terpenes_folder (str): Path to the folder containing input Excel files.
        output_terpenes_folder (str): Path to the folder where the processed output will be saved.
    """
    try:
        # Check if the input folder exists
        if not os.path.exists(input_terpenes_folder):
            logging.error("Input folder does not exist: %s", input_terpenes_folder)
            sys.exit(f"Input folder not found: {input_terpenes_folder}")  # Exit on error
        
        # Check if the output folder exists, create it if it doesn't
        if not os.path.exists(output_terpenes_folder):
            os.makedirs(output_terpenes_folder)
            logging.info("Output folder created: %s", output_terpenes_folder)

        # Loop through all files in the input folder
        for file_name in os.listdir(input_terpenes_folder):
            if file_name.endswith(".xlsx") or file_name.endswith(".xls"):
                file_path = os.path.join(input_terpenes_folder, file_name)
                try:
                    # Load the Excel file and select the sheet named "Mean (%)"
                    df = pd.read_excel(file_path, sheet_name="Mean (ml 100g^-1)")
                    
                    # Get sample names from the first row (excluding the first cell)
                    sample_names = df.columns[1:].tolist()  # Exclude the first column (Chemical Component)
                    
                    # Initialize an empty list to store the output data
                    output_data = []
                    
                    # Loop through each row in the dataframe
                    for _, row in df.iterrows():
                        # The first column is the Chemical Component
                        chemical_component = row.iloc[0]  # First column in the row is the component
                        
                        # Loop through each sample (excluding the first column which is the component)
                        for sample_name in sample_names:
                            concentration = row[sample_name]
                            
                            # Clean the sample name by removing "Mean" and stripping extra spaces
                            cleaned_sample_name = sample_name.replace("Mean", "").strip()
                            
                            # Skip rows where the concentration is 'LOD' or 'LOQ' or 'HLOQ'
                            if pd.notna(concentration) and (concentration != 'LOD' and concentration != 'LOQ' and concentration != 'HLOQ'):
                                # Add the row to output data
                                output_data.append({
                                    "Sample Name": cleaned_sample_name,
                                    "Chemical Component": chemical_component,
                                    "Concentration": concentration
                                })
                    
                    # Create the output DataFrame
                    output_df = pd.DataFrame(output_data)
                    
                    # Save the output DataFrame to a new Excel file
                    output_file_name = f"Output_{file_name}"
                    output_file_path = os.path.join(output_terpenes_folder, output_file_name)
                    output_df.to_excel(output_file_path, index=False)
                    
                    logging.info("Processed file saved as %s", output_file_name)
                    print(f"Processed file saved as {output_file_name}")
                    
                except Exception as e:
                    logging.error("Error processing file %s: %s", file_name, str(e))
                    print(f"Error processing file {file_name}. Check log for details.")
                    sys.exit(f"Error processing file {file_name}. Exiting the system.")  # Exit on error
    
    except Exception as e:
        logging.error("Error in processing files: %s", str(e))
        print("An error occurred. Check the log for details.")
        sys.exit("An error occurred. Exiting the system.")  # Exit on error

# Folder paths
input_terpenes_folder = 'Data/Terpenes/Input' 
output_terpenes_folder = 'Data/Terpenes/Database'  

# Run the processing function
process_terpens_files(input_terpenes_folder, output_terpenes_folder)

Processed file saved as Output_SolventExtraction_100_Dilution1.xlsx
Processed file saved as Output_SolventExtraction_100_Dilution2.xlsx
